In [14]:
import pandas as pd
from sqlalchemy import create_engine
from config import gkey


In [15]:
IPPS=pd.read_csv("IPPS-all.csv")
IPPS=IPPS.loc[IPPS['Provider State'] == 'CA']
IPPS['DRG Definition'] = IPPS['DRG Definition'].str[:3]
IPPS=IPPS[IPPS["DRG Definition"].isin(["233", "234", "235","236"])]
IPPS=IPPS[['DRG Definition',
           'Provider Id', 
           'Provider State',
           'Total Discharges',
           'Average Covered Charges',
           'Average Total Payments',
           'Average Medicare Payments']].copy()
IPPS=IPPS.rename(columns = {'DRG Definition':'drg',
           'Provider Id':'mcrid', 
           'Provider State':'state',
           'Total Discharges':'dc_case',
           'Average Covered Charges':'avg_chg',
           'Average Total Payments':'avg_tbc',
           'Average Medicare Payments':'avg_mcr_chg'})

In [16]:
Cover_CA= pd.read_csv('cover ca.csv')
Cover_CA=Cover_CA[['Entity Name','Medicare Provider Number','OSHPD ID']].copy()
Cover_CA=Cover_CA.dropna()
Cover_CA['Medicare Provider Number'] = Cover_CA['Medicare Provider Number'].astype(int)
Cover_CA['OSHPD ID'] = Cover_CA['OSHPD ID'].astype(int)
Cover_CA=Cover_CA.rename(columns={'Entity Name':'hosp',
                         'Medicare Provider Number':'mcrid',
                         'OSHPD ID':'oshpdid'})

In [17]:
OSHPD= pd.read_csv('OSHPD.csv',encoding = 'unicode_escape')
OSHPD=OSHPD[['Year','Performance Measure','Hospital','OSHPD ID','# of Cases']].copy()
OSHPD=OSHPD.loc[OSHPD['Year']=='2017']
OSHPD.drop([5])
OSHPD['OSHPD ID']=pd.to_numeric(OSHPD['OSHPD ID'], errors='coerce')
OSHPD=OSHPD.rename(columns={'Year':'year',
                            'Performance Measure':'pim',
                            'Hospital':'hosp',
                            'OSHPD ID':'oshpdid',
                            '# of Cases':'num_of_cases'})

In [18]:
engine = create_engine("postgresql://"+gkey)
conn = engine.connect()
engine.table_names()

['ipps', 'cover_ca', 'oshpd']

In [19]:
IPPS.to_sql(name='ipps', con=engine, if_exists='replace', index=False)

In [20]:
Cover_CA.to_sql(name='cover_ca', con=engine, if_exists='replace', index=False)

In [21]:
OSHPD.to_sql(name='oshpd', con=engine, if_exists='replace', index=False)

In [22]:
pd.read_sql_query('select * from ipps',con=engine).head()

,drg,mcrid,state,dc_case,avg_chg,avg_tbc,avg_mcr_chg
0,233,50013,CA,16,594893.0000,68025.93750,66961.93750
1,234,50013,CA,12,455330.7500,44909.91667,44041.91667
2,236,50013,CA,13,487721.8462,42651.15385,41746.76923
3,233,50017,CA,15,659553.1333,86794.80000,85611.66667
4,234,50017,CA,29,428593.6552,47781.37931,46530.24138


In [23]:
pd.read_sql_query('select * from cover_ca',con=engine).head()

,hosp,mcrid,oshpdid
0,BANNER LASSEN MEDICAL CENTER,51320,106184008
1,Orchard Hospital,51311,106040802
2,Colusa Regional Medical Center,50434,106060870
3,EASTERN PLUMAS HOSPITAL-PORTOLA CAMPUS,51300,106320859
4,Fairchild Medical Center,51316,106474007


In [24]:
pd.read_sql_query('select * from oshpd',con=engine).head()

,year,pim,hosp,oshpdid,num_of_cases
0,2017,CABG Operative Mortality,STATEWIDE,NaN,"13,049"
1,2017,CABG Operative Mortality,Alta Bates Summit Medical Center,106010937.0,144
2,2017,CABG Operative Mortality,Valleycare Medical Center,106014050.0,34
3,2017,CABG Operative Mortality,Washington Hospital - Fremont,106010987.0,94
4,2017,CABG Operative Mortality,Enloe Medical Center - Esplanade,106040962.0,155
